In [1]:
import os
os.chdir('..')
os.getcwd()

'/Users/leowyy/Desktop/fyp-graph-clustering'

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt

import pickle 
import numpy as np
import pandas as pd
import torch
import os
from timeit import default_timer as timer
import networkx as nx
import scipy.sparse as sp

In [3]:
from core.SimpleNet import SimpleNet
from core.GraphConvNet import GraphConvNet
from core.EmbeddingDataSet import EmbeddingDataSet
from core.GraphDataBlock import GraphDataBlock
from util.plot_graph_embedding import plot_graph_embedding
from util.evaluation_metrics import evaluate_viz_metrics
from util.network_utils import get_net_projection
from util.graph_utils import get_shortest_path_matrix, neighbor_sampling

cuda not available
cuda not available


In [4]:
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [5]:
if torch.cuda.is_available():
    print('cuda available')
    device = 'cuda'
else:
    print('cuda not available')
    device = 'cpu'

cuda not available


In [6]:
dataset_name = 'reddit_full'
parent_dir = os.path.abspath('..')
input_dir = os.path.join(parent_dir, 'data')
dataset = EmbeddingDataSet(dataset_name, input_dir, train=True)
dataset.create_all_data(n_batches=2, shuffle=False)
dataset.summarise()

Data blocks of length:  [11850, 11849]
Time to create all data (s) = 0.2254
Name of dataset = reddit_full
Input dimension = 602
Number of training samples = 23699
Training labels = True


In [7]:
net_parameters = {}
net_parameters['n_components'] = 2
net_parameters['D'] = dataset.input_dim # input dimension
net_parameters['H'] = 256 # number of hidden units
net_parameters['L'] = 2 # number of hidden layers

In [8]:
n_plot = 70
D_layers = [-1, 0]
mask = np.random.choice(dataset.all_indices, size=n_plot, replace=False)
mask = neighbor_sampling(dataset.adj_matrix, mask, D_layers)
inputs = dataset.inputs[mask]
labels = dataset.labels[mask]
W = dataset.adj_matrix[mask, :][:, mask]
print(len(mask))
G = GraphDataBlock(inputs, labels, W)

959


In [9]:
# Graph net
net_parameters['H'] = 512 # number of hidden units

net_1 = GraphConvNet(net_parameters)
if torch.cuda.is_available(): net_1.cuda()
root = 'results/reddit_full_1/'
filename = root + 'graph_net_1.pkl'
checkpoint = torch.load(filename, map_location=device)
net_1.load_state_dict(checkpoint['state_dict'])

In [10]:
y_emb_1 = get_net_projection(net_1, [G])
plot_graph_embedding(y_emb_1, labels, W, line_alpha=0.1)

In [11]:
# Graph net
net_parameters['H'] = 512 # number of hidden units

net_1 = GraphConvNet(net_parameters)
if torch.cuda.is_available(): net_1.cuda()
root = 'results/reddit_full_2/'
filename = root + 'graph_net_1.pkl'
checkpoint = torch.load(filename, map_location=device)
net_1.load_state_dict(checkpoint['state_dict'])

In [12]:
y_emb_2 = get_net_projection(net_1, [G])
plot_graph_embedding(y_emb_2, labels, W, line_alpha=0.1)

In [13]:
# Graph net
net_parameters['H'] = 512 # number of hidden units

net_1 = GraphConvNet(net_parameters)
if torch.cuda.is_available(): net_1.cuda()
root = 'results/reddit_full_3/'
filename = root + 'graph_net_1.pkl'
checkpoint = torch.load(filename, map_location=device)
net_1.load_state_dict(checkpoint['state_dict'])

In [14]:
y_emb_3 = get_net_projection(net_1, [G])
plot_graph_embedding(y_emb_3, labels, W, line_alpha=0.1)

In [15]:
# dw_embed = y_deepwalk_256[mask]
# from sklearn.manifold import TSNE
# embedder = TSNE(n_components=2, method="exact", perplexity=30, verbose=1)
# y_pred_deepwalk = embedder.fit_transform(dw_embed)

In [16]:
# plot_graph_embedding(y_pred_deepwalk, labels, W, line_alpha=0.1)

In [17]:
# path_matrix = get_shortest_path_matrix(W.toarray())
# path_matrix.shape

In [18]:
# from sklearn.manifold import TSNE
# embedder = TSNE(n_components=2, metric='precomputed', method="exact", perplexity=30, verbose=1)
# y_pred_tsne = embedder.fit_transform(path_matrix)

In [19]:
# plot_graph_embedding(y_pred_tsne, labels, W, line_alpha=0.1)

In [20]:
_ = evaluate_viz_metrics(y_emb_1, G)

Metric one_nn = 0.1888
Metric five_nn = 0.2034
Metric graph_dist = 2.5768
Metric feature_dist = 0.1504
Metric total_dist = 2.7273


In [21]:
_ = evaluate_viz_metrics(y_emb_2, G)

Metric one_nn = 0.6997
Metric five_nn = 0.7497
Metric graph_dist = 0.1928
Metric feature_dist = 1.9572
Metric total_dist = 2.1499


In [22]:
_ = evaluate_viz_metrics(y_emb_3, G)

Metric one_nn = 0.2305
Metric five_nn = 0.2764
Metric graph_dist = 1.8481
Metric feature_dist = 0.4019
Metric total_dist = 2.2500


In [23]:
# _ = combined_metric(y_pred_deepwalk, inputs, W, k=5)